<a id="top"></a>
# NARCliM 2.0 WA Climate Explorer

This notebook provides an interactive interface to explore NARCliM 2.0 monthly climate data over Western Australia using data served from NCI's THREDDS server. It is intended as a reusable exploration tool rather than a production workflow.

## Quick start

> 1. Run the single Python cell below.  
> 2. Use the panel at the bottom to pick a **Variable** and **Year**.  
> 3. Click **🚀 ANALYZE**, then select regions and click **📈 TIMESERIES**.
>
> You do not need to run or edit any other cells.


In [ ]:
# NARCliM 2.0 WA Climate Explorer – launcher cell

from narclim_wa_explorer import build_wa_explorer_ui

ui = build_wa_explorer_ui()
display(ui)


## WA Climate Explorer controls

Use the controls to load and visualise NARCliM 2.0 data:

- **Variable**: choose a climate variable (e.g. tasmax).  
- **Year**: choose one of the years available in the THREDDS catalog for that variable.  
- **🚀 ANALYZE**: loads data for that year, clips to WA, computes mean/min/max, and draws the interactive map.  
- **WA regions list + 📈 TIMESERIES**: plots time series at the nearest grid point to each selected region.


<div style="text-align: right">
  <a href="#top">Go back to top</a>
</div>
